In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

#Load the dataset
dataset = pd.read_csv('obesity_dataset.csv')

#Preprocessing
label_enc = LabelEncoder()
categorical_cols = ['Gender', 'CALC', 'FAVC', 'SCC', 'SMOKE',
                    'family_history_with_overweight', 'CAEC', 'MTRANS', 'NObeyesdad']
for col in categorical_cols:
    dataset[col] = label_enc.fit_transform(dataset[col])

# Separate features (X) and target variable (y)
X = dataset.drop('NObeyesdad', axis=1)
y = dataset['NObeyesdad']

# Normalize numerical features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

#Initialize the models
models = {
    'kNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=1000)
}

#Train and validate each model
results = {}
for model_name, model in models.items():
    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

    # Train the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Compute test accuracy
    test_accuracy = accuracy_score(y_test, y_pred)

    # Store the results
    results[model_name] = {
        'Cross-Validation Mean Accuracy': cv_scores.mean(),
        'Test Accuracy': test_accuracy,
        'Classification Report': classification_report(y_test, y_pred, output_dict=True)
    }

#Display the results in a DataFrame for better readability
results_df = pd.DataFrame(results).T

#Display the results
import ace_tools as tools
tools.display_dataframe_to_user(name="Model Training and Validation Results", dataframe=results_df)
